In [2]:
%load_ext autoreload
%autoreload 2

In [6]:
import unittest
import pandas as pd
import numpy as np

import os
import inspect
import sys
from pandas._testing import assert_frame_equal


sys.path.insert(1, '../src/preprocessing/')
from preprocessing_lib import nn_format
from dataset_lib import shift


In [7]:
class TestCreateTimeseries(unittest.TestCase):
    
    def no_test_nn_format(self):
        df = pd.DataFrame({
            "start_time": ["2018-01-01 00:00:00"] * 3 + ["2018-01-01 01:00:00"] * 3 +  ["2018-01-01 02:00:00"] * 3,
            "from_station_id": [1, 2, 3, 1, 2, 3, 1, 2, 3],
            "quantity": [100,200,300,101,201,301, 102, 202, 302]
        })
        # df = df.reset_index(drop=True).set_index("start_time")

        expected_df = pd.DataFrame({ 
            "start_time": ["2018-01-01 00:00:00", "2018-01-01 01:00:00", "2018-01-01 02:00:00"],
            "quantity_1": [100, 101, 102],
            "quantity_2": [200, 201, 202],
            "quantity_3": [300, 301, 302],
        })
        expected_df = expected_df.reset_index(drop=True).set_index("start_time")

        df_nn = nn_format(df)
        assert_frame_equal(expected_df, df_nn)
    
    def test_shift_by_one(self):
        df = pd.DataFrame(pd.DataFrame({
            "start_time": ["2018-01-01 00:00:00", "2018-01-01 01:00:00", "2018-01-01 02:00:00",
                        "2018-01-02 00:00:00", "2018-01-02 01:00:00", "2018-01-02 02:00:00",
                        "2018-01-03 00:00:00", "2018-01-03 01:00:00", "2018-01-03 02:00:00",
                        "2018-01-04 00:00:00", "2018-01-04 01:00:00", "2018-01-04 02:00:00"],
            "hour": [1, 2, 3] * 4,
            "day_of_week": [5] * 3 + [6] * 3 + [7] * 3 + [1] * 3,
            "month": [1] * 12,
            "quantity_1": [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111],
            "quantity_2": [200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211],
            "quantity_3": [300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311],
        }))
        df["start_time"] = pd.to_datetime(
                df["start_time"], format='%Y-%m-%d %H:%M:%S')
        df = df.reset_index(drop=True).set_index("start_time")

        expected_df = pd.DataFrame(pd.DataFrame({
            "start_time": ["2018-01-02 00:00:00", "2018-01-02 01:00:00", "2018-01-02 02:00:00",
                        "2018-01-03 00:00:00", "2018-01-03 01:00:00", "2018-01-03 02:00:00",
                        "2018-01-04 00:00:00", "2018-01-04 01:00:00", "2018-01-04 02:00:00"],
            "hour": [1, 2, 3] * 3,
            "day_of_week": [6] * 3 + [7] * 3 + [1] * 3,
            "month": [1] * 9,
            "quantity_1": [100, 101, 102, 103, 104, 105, 106, 107, 108],
            "quantity_2": [200, 201, 202, 203, 204, 205, 206, 207, 208],
            "quantity_3": [300, 301, 302, 303, 304, 305, 306, 307, 308],
        }))
        expected_df["start_time"] = pd.to_datetime(
                expected_df["start_time"], format='%Y-%m-%d %H:%M:%S')
        expected_df = expected_df.reset_index(drop=True).set_index("start_time")
        assert_frame_equal(expected_df, shift(df))


        

In [8]:
if __name__ == '__main__':
    unittest.main(argv=[""], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.019s

OK
